d ##### Usecase:Get sales data by customers
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for customers per day
###### Problem statement-2:Get sales data for customers per month
###### Problem statement-3:Get sales date for customers per year
###### Problem statement-4:Get monthly sales data from most visited customers
###### Problem statement-5:Get monthly sales data from Top10 revenue generating customers 
###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where customer_state is CA
###### Problem statement-7:Get monthly sales data from Top10 revenue generating cities
###### Problem statement-8:Get Top10 revenue generating zipcodes and its corresponding cities and states
###### Problem statement-9:Get revenue of  each zipcodes of state COLORADO and its corresponding cities 
###### Problem statement-10:Get total revenue of each state in last quarter of 2013

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count,substring
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[1]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database retail_db")
hiveContext.sql("use retail_db")

In [4]:
hiveContext.sql("create table orders(order_id int,order_date string,order_customer_id int, order_status string) row format delimited fields terminated by ','")
hiveContext.sql("create table order_items(order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float) row format delimited fields terminated by ','")
hiveContext.sql("create table customers(customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, customer_state string, customer_zipcode string) row format delimited fields terminated by ','")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Orders.txt' into table orders")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Order_Items.txt' into table order_items")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Customers.txt' into table customers")

In [6]:
hiveContext.sql("show tables").show()

+---------+-----------+-----------+
 database| tableName|isTemporary|
+---------+-----------+-----------+
retail_db| categories| false|
retail_db| customers| false|
retail_db|departments| false|
retail_db|order_items| false|
retail_db| orders| false|
retail_db| products| false|
+---------+-----------+-----------+

In [7]:
hiveContext.sql("select * from orders").show(5,False)

+--------+-----------------------+-----------------+---------------+
order_id|order_date |order_customer_id|order_status |
+--------+-----------------------+-----------------+---------------+
1 |2013-07-25 00:00:00.000|11599 |CLOSED |
2 |2013-07-25 00:00:00.000|256 |PENDING_PAYMENT|
3 |2013-07-25 00:00:00.000|12111 |COMPLETE |
4 |2013-07-25 00:00:00.000|8827 |CLOSED |
5 |2013-07-25 00:00:00.000|11318 |COMPLETE |
+--------+-----------------------+-----------------+---------------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from orders").printSchema()

root
-- order_id: string (nullable = true)
-- order_date: string (nullable = true)
-- order_customer_id: string (nullable = true)
-- order_status: string (nullable = true)

In [9]:
hiveContext.sql("select * from order_items").show(5,False)


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [10]:
hiveContext.sql("select * from order_items").printSchema()

root
-- order_item_id: string (nullable = true)
-- order_item_order_id: string (nullable = true)
-- order_item_product_id: string (nullable = true)
-- order_item_quantity: string (nullable = true)
-- order_item_subtotal: string (nullable = true)
-- order_item_product_price: string (nullable = true)

In [11]:
hiveContext.sql("select * from customers").show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [12]:
hiveContext.sql("select * from customers").printSchema()

root
-- customer_id: string (nullable = true)
-- customer_fname: string (nullable = true)
-- customer_lname: string (nullable = true)
-- customer_email: string (nullable = true)
-- customer_password: string (nullable = true)
-- customer_street: string (nullable = true)
-- customer_city: string (nullable = true)
-- customer_state: string (nullable = true)
-- customer_zipcode: string (nullable = true)

###### Problem statement-1:Get sales data for customers per day

In [14]:
customersSalesDataPerDay = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,ctl.customer_fname as Customer_firstname,ctl.customer_lname as Customer_lastname,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perday,  
                                   count(oitl.order_item_subtotal) as NumberOforders_perday, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by orderdate,ctl.customer_fname, ctl.customer_lname 
                                   order by orderdate,ctl.customer_fname, ctl.customer_lname,NumberOforders_perday,Totalrevenue_perday''')
customersSalesDataPerDay.coalesce(2).write.csv("/FileStore/tables/customersSalesDataPerDay", compression="none", header ='true')
customersSalesDataPerDay.show(20,False)

+----------+------------------+-----------------+-------------------+---------------------+---------------+
orderdate |Customer_firstname|Customer_lastname|Totalrevenue_perday|NumberOforders_perday|avgsales_perday|
+----------+------------------+-----------------+-------------------+---------------------+---------------+
2013-07-25|Albert |Jones |199.99 |1 |199.99 |
2013-07-25|Alice |Rodriguez |372.91 |4 |93.23 |
2013-07-25|Amy |Cummings |119.97 |1 |119.97 |
2013-07-25|Amy |Smith |309.95 |3 |103.32 |
2013-07-25|Brian |Wilson |699.85 |4 |174.96 |
2013-07-25|Charles |Smith |327.88 |2 |163.94 |
2013-07-25|Christian |Smith |499.85 |4 |124.96 |
2013-07-25|Christina |Lewis |829.86 |3 |276.62 |
2013-07-25|Danielle |Moran |299.98 |1 |299.98 |
2013-07-25|David |Kerr |879.86 |4 |219.97 |
2013-07-25|David |Rodriguez |579.98 |3 |193.33 |
2013-07-25|Deborah |Barber |499.94 |2 |249.97 |
2013-07-25|Diane |Davis |739.92 |3 |246.64 |
2013-07-25|Diane |Wiggins |229.95 |2 |114.98 |
2013-07-25|Donna |Williams |463.92 |4 |115.98 |
2013-07-25|Doris |Smith |899.94 |4 |224.99 |
2013-07-25|Douglas |Smith |549.94 |3 |183.31 |
2013-07-25|Edward |Castaneda |549.95 |3 |183.32 |
2013-07-25|Eric |Smith |629.96 |3 |209.99 |
2013-07-25|Evelyn |Herring |199.92 |1 |199.92 |
+----------+------------------+-----------------+-------------------+---------------------+---------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for customers per month

In [16]:
customerSalesDataPerMonth= hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,ctl.customer_fname as Customer_firstname,ctl.customer_lname as Customer_lastname,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_permonth,  
                                   count(oitl.order_item_subtotal) as NumberOforders_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by ordermonth,ctl.customer_fname, ctl.customer_lname 
                                   order by ordermonth,ctl.customer_fname, ctl.customer_lname,NumberOforders_permonth,Totalrevenue_permonth''')
customerSalesDataPerMonth.coalesce(2).write.csv("/FileStore/tables/customerSalesDataPerMonth", compression="none", header ='true')
customerSalesDataPerMonth.show(20,False)

+----------+------------------+-----------------+---------------------+-----------------------+-----------------+
ordermonth|Customer_firstname|Customer_lastname|Totalrevenue_permonth|NumberOforders_permonth|avgsales_permonth|
+----------+------------------+-----------------+---------------------+-----------------------+-----------------+
2013-07 |Aaron |Hanna |1169.79 |5 |233.96 |
2013-07 |Aaron |Hudson |419.92 |3 |139.97 |
2013-07 |Aaron |Smith |799.91 |4 |199.98 |
2013-07 |Adam |Marquez |669.98 |3 |223.33 |
2013-07 |Alan |Gillespie |919.94 |4 |229.99 |
2013-07 |Alan |Weber |999.9 |5 |199.98 |
2013-07 |Albert |Jones |199.99 |1 |199.99 |
2013-07 |Alexander |Dominguez |716.83 |5 |143.37 |
2013-07 |Alexander |Nguyen |959.91 |4 |239.98 |
2013-07 |Alexander |Smith |299.98 |1 |299.98 |
2013-07 |Alexander |Williams |499.98 |2 |249.99 |
2013-07 |Alice |Barnes |669.9 |3 |223.3 |
2013-07 |Alice |Rodriguez |372.91 |4 |93.23 |
2013-07 |Alice |Smith |2089.73 |10 |208.97 |
2013-07 |Amanda |Brown |99.96 |1 |99.96 |
2013-07 |Amanda |Foster |1459.88 |6 |243.31 |
2013-07 |Amanda |Gibson |1029.93 |4 |257.48 |
2013-07 |Amanda |Hays |1059.81 |5 |211.96 |
2013-07 |Amanda |Shaffer |499.94 |2 |249.97 |
2013-07 |Amanda |Smith |799.98 |3 |266.66 |
+----------+------------------+-----------------+---------------------+-----------------------+-----------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for customers per year

In [18]:
customerSalesDataPerYear = hiveContext.sql('''select substring(otl.order_date,0,4) as orderyear,ctl.customer_fname as Customer_firstname,ctl.customer_lname as Customer_lastname,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_peryear,  
                                   count(oitl.order_item_subtotal) as NumberOforders_peryear, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_peryear 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by orderyear,ctl.customer_fname, ctl.customer_lname 
                                   order by orderyear,ctl.customer_fname, ctl.customer_lname,NumberOforders_peryear,Totalrevenue_peryear''')
customerSalesDataPerYear.coalesce(2).write.csv("/FileStore/tables/customerSalesDataPerYear", compression="none", header ='true')
customerSalesDataPerYear.show(20,False)

+---------+------------------+-----------------+--------------------+----------------------+----------------+
orderyear|Customer_firstname|Customer_lastname|Totalrevenue_peryear|NumberOforders_peryear|avgsales_peryear|
+---------+------------------+-----------------+--------------------+----------------------+----------------+
2013 |Aaron |Berger |1929.72 |9 |214.41 |
2013 |Aaron |Boyle |679.98 |6 |113.33 |
2013 |Aaron |Bush |1119.89 |4 |279.97 |
2013 |Aaron |Carr |1969.71 |11 |179.06 |
2013 |Aaron |Cline |1794.89 |7 |256.41 |
2013 |Aaron |Davis |989.92 |5 |197.98 |
2013 |Aaron |Hanna |3079.47 |18 |171.08 |
2013 |Aaron |Hudson |4559.49 |18 |253.3 |
2013 |Aaron |Kennedy |604.91 |4 |151.23 |
2013 |Aaron |Mann |99.99 |1 |99.99 |
2013 |Aaron |Mcmahon |699.88 |3 |233.29 |
2013 |Aaron |Norman |979.92 |5 |195.98 |
2013 |Aaron |Riddle |2544.81 |11 |231.35 |
2013 |Aaron |Salazar |725.88 |5 |145.18 |
2013 |Aaron |Sharp |1099.92 |5 |219.98 |
2013 |Aaron |Smith |14410.4 |76 |189.61 |
2013 |Aaron |Valdez |1199.94 |3 |399.98 |
2013 |Aaron |Villa |1399.8 |8 |174.98 |
2013 |Aaron |Williamson |419.97 |3 |139.99 |
2013 |Adam |Blair |3354.68 |11 |304.97 |
+---------+------------------+-----------------+--------------------+----------------------+----------------+
only showing top 20 rows

###### Problem statement-4:Get monthly sales data from customers visited more than 100 times

In [20]:
customerSalesDataPerMonth = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,ctl.customer_fname as Customerfirstname,ctl.customer_lname as Customerlastname,
                                   count(otl.order_customer_id) as customervisit_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by ordermonth,ctl.customer_fname, ctl.customer_lname
                                   having customervisit_permonth>=100
                                   order by ordermonth,ctl.customer_fname, ctl.customer_lname,customervisit_permonth''')
customerSalesDataPerMonth.coalesce(2).write.csv("/FileStore/tables/customerSalesDataPerMonth", compression="none", header ='true')
customerSalesDataPerMonth.show(20,False)

+----------+-----------------+----------------+----------------------+
ordermonth|Customerfirstname|Customerlastname|customervisit_permonth|
+----------+-----------------+----------------+----------------------+
2013-07 |Mary |Smith |545 |
2013-08 |Mary |Smith |2048 |
2013-09 |Mary |Smith |2063 |
2013-10 |Mary |Smith |1833 |
2013-11 |Mary |Smith |2224 |
2013-12 |James |Smith |101 |
2013-12 |Mary |Smith |1929 |
2014-01 |Mary |Smith |2040 |
2014-02 |Mary |Smith |1956 |
2014-03 |Mary |Smith |1969 |
2014-03 |Robert |Smith |108 |
2014-04 |Mary |Smith |1948 |
2014-05 |Mary |Smith |1974 |
2014-06 |Mary |Smith |1790 |
2014-07 |Mary |Smith |1565 |
+----------+-----------------+----------------+----------------------+

###### Problem statement-5:Get monthly sales data from Top10 revenue generating customers

In [22]:
customeDataTop10revenue = hiveContext.sql('''select ctl.customer_fname as Customerfirstname,ctl.customer_lname as Customerlastname,
                                   round(sum(oitl.order_item_subtotal),2) as customer_revenue
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by ctl.customer_fname, ctl.customer_lname 
                                   order by customer_revenue desc limit 10''')
customeDataTop10revenue.coalesce(2).write.csv("/FileStore/tables/customeDataTop10revenue", compression="none", header ='true')
customeDataTop10revenue.show(20,False)

+-----------------+----------------+----------------+
Customerfirstname|Customerlastname|customer_revenue|
+-----------------+----------------+----------------+
Mary |Smith |4771645.79 |
Robert |Smith |152230.96 |
James |Smith |147924.43 |
David |Smith |145776.8 |
John |Smith |129882.58 |
William |Smith |108523.28 |
Michael |Smith |92843.03 |
Elizabeth |Smith |90516.02 |
Mary |Jones |88860.48 |
Christopher |Smith |85271.05 |
+-----------------+----------------+----------------+

###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where and Numberofsales>=110 in state of CA

In [24]:
customerSalesDataCAstate = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,ctl.customer_state,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue,
                                   count(oitl.order_item_subtotal) as Numberofsales, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   OR( CAST(otl.order_date AS DATE)>='2014-04-01' AND CAST(otl.order_date AS DATE)<='2014-06-30')
                                   group by orderdate,ctl.customer_state 
                                   having ctl.customer_state = 'CA' and Numberofsales>=110
                                   order by orderdate''')
customerSalesDataCAstate.coalesce(1).write.csv("/FileStore/tables/customerSalesDataCAstate", compression="none", header ='true')
customerSalesDataCAstate.show(20,False)

+----------+--------------+------------+-------------+--------+
orderdate |customer_state|Totalrevenue|Numberofsales|avgsales|
+----------+--------------+------------+-------------+--------+
2013-10-06|CA |26038.14 |138 |188.68 |
2013-11-03|CA |35462.01 |172 |206.17 |
2013-11-05|CA |25370.88 |124 |204.6 |
2013-11-07|CA |23501.35 |113 |207.98 |
2013-11-11|CA |25539.55 |135 |189.18 |
2013-11-14|CA |22587.48 |111 |203.49 |
2013-11-23|CA |28184.84 |137 |205.73 |
2013-11-24|CA |26089.8 |135 |193.26 |
2013-12-18|CA |29707.34 |150 |198.05 |
2013-12-20|CA |21190.68 |110 |192.64 |
2013-12-21|CA |21396.22 |110 |194.51 |
2013-12-26|CA |30697.02 |154 |199.33 |
2014-04-21|CA |21622.29 |112 |193.06 |
2014-04-23|CA |24004.27 |124 |193.58 |
2014-05-02|CA |21322.58 |111 |192.1 |
2014-05-12|CA |24708.23 |113 |218.66 |
2014-06-02|CA |23938.49 |115 |208.16 |
2014-06-08|CA |20986.06 |110 |190.78 |
2014-06-09|CA |21992.26 |125 |175.94 |
+----------+--------------+------------+-------------+--------+

###### Problem statement-7:Get Top10 revenue generating cities

In [26]:
customeDataTop10revcities = hiveContext.sql('''select ctl.customer_city, 
                                   round(sum(oitl.order_item_subtotal),2) as city_revenue
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by ctl.customer_city
                                   order by city_revenue desc limit 10''')
customeDataTop10revcities.coalesce(1).write.csv("/FileStore/tables/customeDataTop10revcities", compression="none", header ='true')
customeDataTop10revcities.show(20,False)

+-------------+-------------+
customer_city|city_revenue |
+-------------+-------------+
Caguas |1.269771807E7|
Chicago |746869.57 |
Los Angeles |645083.23 |
Brooklyn |643507.92 |
New York |338937.57 |
Philadelphia |297670.28 |
Bronx |288041.71 |
San Diego |279678.06 |
Miami |253232.67 |
Houston |248282.44 |
+-------------+-------------+

###### Problem statement-8:Get  Top10 revenue generating zipcodes and its corresponding cities and states

In [28]:
customeDataTop10ziprevenue = hiveContext.sql('''select ctl.customer_zipcode,ctl.customer_city,ctl.customer_state,  
                                   round(sum(oitl.order_item_subtotal),2) as zipcode_revenue
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   group by ctl.customer_zipcode,ctl.customer_city,ctl.customer_state
                                   order by zipcode_revenue desc limit 10''')
customeDataTop10ziprevenue.coalesce(1).write.csv("/FileStore/tables/customeDataTop10ziprevenue", compression="none", header ='true')
customeDataTop10ziprevenue.show(20,False)

+----------------+---------------+--------------+---------------+
customer_zipcode|customer_city |customer_state|zipcode_revenue|
+----------------+---------------+--------------+---------------+
00725 |Caguas |PR |1.269771807E7 |
23455 |Virginia Beach |VA |66357.08 |
33324 |Fort Lauderdale|FL |57030.12 |
79109 |Amarillo |TX |55147.6 |
92024 |Encinitas |CA |50634.74 |
33624 |Tampa |FL |50504.17 |
78586 |San Benito |TX |50280.56 |
11226 |Brooklyn |NY |49229.64 |
80012 |Aurora |CO |49203.59 |
92115 |San Diego |CA |48850.26 |
+----------------+---------------+--------------+---------------+

###### Problem statement-9:Get revenue of  each zipcodes of state COLORADO and its corresponding cities

In [30]:
zipcoderevenueDataCOstate = hiveContext.sql('''select ctl.customer_zipcode,ctl.customer_city,ctl.customer_state,  
                                   round(sum(oitl.order_item_subtotal),2) as zipcode_revenue
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   where ctl.customer_state='CO'
                                   group by ctl.customer_state,ctl.customer_zipcode,ctl.customer_city
                                   order by ctl.customer_zipcode''')
zipcoderevenueDataCOstate.coalesce(1).write.csv("/FileStore/tables/zipcoderevenueDataCOstate", compression="none", header ='true')
zipcoderevenueDataCOstate.show(30,False)

+----------------+----------------+--------------+---------------+
customer_zipcode|customer_city |customer_state|zipcode_revenue|
+----------------+----------------+--------------+---------------+
80010 |Aurora |CO |21922.41 |
80011 |Aurora |CO |22319.24 |
80012 |Aurora |CO |49203.59 |
80013 |Aurora |CO |24281.22 |
80015 |Aurora |CO |16424.21 |
80020 |Broomfield |CO |12344.44 |
80110 |Englewood |CO |13481.41 |
80126 |Littleton |CO |27356.53 |
80219 |Denver |CO |29316.54 |
80231 |Denver |CO |16529.28 |
80501 |Longmont |CO |37619.57 |
80631 |Greeley |CO |40771.26 |
80906 |Colorado Springs|CO |22490.68 |
80918 |Colorado Springs|CO |24250.22 |
+----------------+----------------+--------------+---------------+

###### Problem statement-10:Get total revenue of each state in last quarter of 2013

In [32]:
SalesDatarevenueperstate = hiveContext.sql('''select ctl.customer_state,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue,
                                   count(oitl.order_item_subtotal) as Numberofsales, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                   join customers ctl on otl.order_customer_id = ctl.customer_id
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   group by ctl.customer_state 
                                   order by ctl.customer_state''')
SalesDatarevenueperstate.coalesce(1).write.csv("/FileStore/tables/SalesDatarevenueperstate", compression="none", header ='true')
SalesDatarevenueperstate.show(51,False)

+--------------+------------+-------------+--------+
customer_state|Totalrevenue|Numberofsales|avgsales|
+--------------+------------+-------------+--------+
AL |399.98 |1 |399.98 |
AR |4439.46 |25 |177.58 |
AZ |140367.09 |699 |200.81 |
CA |1400872.48 |7078 |197.92 |
CO |100241.18 |517 |193.89 |
CT |63555.89 |307 |207.02 |
DC |27194.55 |141 |192.87 |
DE |11263.62 |54 |208.59 |
FL |272265.35 |1372 |198.44 |
GA |110664.95 |558 |198.32 |
HI |58084.99 |295 |196.9 |
IA |4319.66 |21 |205.7 |
ID |8853.92 |41 |215.95 |
IL |375566.27 |1845 |203.56 |
IN |27682.43 |139 |199.15 |
KS |16973.94 |83 |204.51 |
KY |28011.74 |137 |204.47 |
LA |47249.39 |226 |209.07 |
MA |74428.63 |361 |206.17 |
MD |123918.13 |624 |198.59 |
MI |190619.7 |955 |199.6 |
MN |29691.6 |140 |212.08 |
MO |70390.18 |352 |199.97 |
MT |4634.39 |22 |210.65 |
NC |107873.51 |555 |194.37 |
ND |9339.94 |53 |176.23 |
NJ |160767.13 |814 |197.5 |
NM |42035.25 |214 |196.43 |
NV |63332.86 |321 |197.3 |
NY |538938.82 |2733 |197.2 |
OH |212179.54 |1065 |199.23 |
OK |15272.61 |78 |195.8 |
OR |73191.64 |369 |198.35 |
PA |193118.7 |972 |198.68 |
PR |3351447.58 |16831 |199.12 |
RI |11333.42 |57 |198.83 |
SC |33575.15 |164 |204.73 |
TN |81113.7 |418 |194.05 |
TX |418383.75 |2066 |202.51 |
UT |41741.71 |203 |205.62 |
VA |92069.8 |481 |191.41 |
WA |43608.15 |230 |189.6 |
WI |34977.33 |176 |198.73 |
WV |10230.8 |57 |179.49 |
+--------------+------------+-------------+--------+